# Excercise 5: Deploy our model to the Cloud



## Configure gcloud and kukbectl

We now authenticate against our cloud cluster. Open a Jupyter Terminal and start authenticating:

`gcloud init --console-only`

Confirm that you want to login and copy the URL to your browser (**using right-click | Copy**). Loging using your google account, confirm permissions and copy the acccess token (**using right-click | Paste**).

If asked select project **academy-300812**




## Check Connection

To connect to the cluster execute this:

`gcloud container clusters get-credentials ml-in-production --zone europe-west3-c --project academy-300812`  

After executing this you should be able to see our cluster nodes:

`kubectl get nodes`

should output something like

```
NAME                                              STATUS   ROLES    AGE   VERSION
gke-ml-in-production-default-pool-2167de87-40d0   Ready    <none>   22h   v1.18.16-gke.2100
```



## Prepare deployment

Create a namespace for you: `kubectl create namespace <yourname>`

And configure it as the default namespace :
`kubectl config set-context $(kubectl config current-context) --namespace=<yourname>`



## Deploy application

Now we are able to deploy your sustaino service. Configuration YAMLs are provided in the `pipeline` folder.

1. Have a look at frontend.yml and eligibility-serivce.yml
2. change to the work directory `cd work`
3. Apply the configuration to the cluster: `kubectl apply -f pipeline`
4. Wait for alle PODs to be deployed: `kubectl get pods`
   
If all pods are in the state _running_ our application was successfully deployed.





## Access Application

To access the application we need the IP address of the provisioned load balancer. 

You get this IP by executing `kubectl get ingress`

Copy the IP address and open it in a browser. It can take up to 10 minutes for the load balancer to configure correctly. So if you see a 404 or 502 error page, just wait a little bit.

## Execute Pipeline

A Cloud Build Pipeline is configured in the file `cloudbuild.yaml`. It executes a training script at `pipeline/train_model.py` and builds a Docker image in the cloud.

1. Read through `pipeline/train_model.py` most of it should be very similar.
2. Replace the placeholder `<yourname>` in `cloudbuild.yaml`.
3. In order to prevent permission errors when building, execute `chmod 777 -R ~/work/services/eligibility` 
3. Build our model in the cloud: execute `gcloud builds submit`

## Configure our ML service

After our cloud build ran successfully, we have a model packaged in a docker image stored in the cloud. To deploy it reference your image in `pipeline/eligibility-service.yml`

Update `image: fluescher/sustaino-dummy-service` to `image: gcr.io/academy-300812/<yourname>/eligibility-service:latest`

and reapply the files again `kubectl apply -f pipeline`

Test our sustaino frontend. Our model should be used now.
